In [2]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

from huggingface_hub import login


In [3]:
login() 

# InferenceClient

InferenceClient aims to provide a unified experience to perform inference. The client can be used seamlessly with either the (free) Inference API, self-hosted Inference Endpoints, or third-party Inference Providers. https://huggingface.co/docs/huggingface_hub/v0.28.1/en/package_reference/inference_client#huggingface_hub.InferenceClient.chat_completion

* Modelo de meta

In [4]:
from huggingface_hub import InferenceClient

client = InferenceClient("meta-llama/Meta-Llama-3-8B-Instruct")

In [8]:
messages = [{"role": "user", "content": "What is the capital of France?"}]
response = client.chat_completion(messages, max_tokens=100)

print(response)
print("response:",response.choices[0].message.content)

ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop', index=0, message=ChatCompletionOutputMessage(role='assistant', content='The capital of France is Paris.', tool_calls=None), logprobs=None)], created=1739358178, id='', model='meta-llama/Meta-Llama-3-8B-Instruct', system_fingerprint='3.0.1-sha-bb9095a', usage=ChatCompletionOutputUsage(completion_tokens=8, prompt_tokens=17, total_tokens=25))
response: The capital of France is Paris.


* Modelo de Qwen2.5-Coder

In [9]:
from huggingface_hub import InferenceClient

qwen_client = InferenceClient("Qwen/Qwen2.5-Coder-1.5B-Instruct")

In [11]:
messages = [
    {"role": "system", "content": "You are expert in SQL and can translate natural language to SQL"},
    {"role": "user", "content": "Show me the total number of rows from the inventory table"}
]

response = qwen_client.chat_completion(messages)

print(response)
print("response:",response.choices[0].message.content)

ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop', index=0, message=ChatCompletionOutputMessage(role='assistant', content='To show the total number of rows from the inventory table, you can use the following SQL query:\n\n```sql\nSELECT COUNT(*) AS total_rows\nFROM inventory;\n```\n\nThis query will return a result set with one column and one row, where the value of the `total_rows` column will be the total number of rows in the inventory table.', tool_calls=None), logprobs=None)], created=1739358587, id='', model='Qwen/Qwen2.5-Coder-1.5B-Instruct', system_fingerprint='3.0.1-sha-bb9095a', usage=ChatCompletionOutputUsage(completion_tokens=72, prompt_tokens=36, total_tokens=108))
response: To show the total number of rows from the inventory table, you can use the following SQL query:

```sql
SELECT COUNT(*) AS total_rows
FROM inventory;
```

This query will return a result set with one column and one row, where the value of the `total_rows` column will be th

* Modelo de T5-LM-Large-text2sql-spider

In [12]:
from huggingface_hub import InferenceClient

spider_client = InferenceClient("gaussalgo/T5-LM-Large-text2sql-spider")

In [ ]:
messages = [
    {"role": "system", "content": "You are expert in SQL and can translate natural language to SQL"},
    {"role": "user", "content": "Show me the total number of rows from the inventory table"}
]

response = spider_client.chat_completion(messages)

print(response)
print("response:",response.choices[0].message.content)
# 400 Client Error: Bad Request for url

#  Chat

In [55]:

llm = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

chat_model = ChatHuggingFace(llm=llm)

In [56]:
chat_model.invoke("Hello, how are you?")

AIMessage(content="I am not capable of experiencing emotions or sensations like humans do. However, I am programmed to respond in a polite and friendly manner. So, I'm doing well, thank you for asking. How about you?", additional_kwargs={}, response_metadata={'token_usage': ChatCompletionOutputUsage(completion_tokens=47, prompt_tokens=28, total_tokens=75), 'model': '', 'finish_reason': 'stop'}, id='run-b4714fb5-b95c-4e7e-93e1-9d4ec7f95b81-0')

# Summarization

In [57]:
customer_email = """
I hope this email finds you amidst an aura of understanding, despite the tangled mess of emotions swirling within me as I write to you. I am writing to pour my heart out about the recent unfortunate experience I had with one of your coffee machines that arrived ominously broken, evoking a profound sense of disbelief and despair.

To set the scene, let me paint you a picture of the moment I anxiously unwrapped the box containing my highly anticipated coffee machine. The blatant excitement coursing through my veins could rival the vigorous flow of coffee through its finest espresso artistry. However, what I discovered within broke not only my spirit but also any semblance of confidence I had placed in your esteemed brand.

Imagine, if you can, the utter shock and disbelief that took hold of me as I laid eyes on a disheveled and mangled coffee machine. Its once elegant exterior was marred by the scars of travel, resembling a war-torn soldier who had fought valiantly on the fields of some espresso battlefield. This heartbreaking display of negligence shattered my dreams of indulging in daily coffee perfection, leaving me emotionally distraught and inconsolable
"""  # created by GPT-3.5

In [58]:

from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
)

messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(
        content=f"Summarize this: {customer_email}"
    ),
]


chat_model.invoke(messages)



AIMessage(content="The writer is sharing their disappointment about a coffee machine they received that arrived broken, despite their high expectations. They describe feeling a mix of disbelief, despair, and emotional distress upon discovering the damaged machine, as its appearance reminded them of a damaged soldier. The writer's initial excitement about using the machine was shattered, leaving them feeling upset and disappointed.", additional_kwargs={}, response_metadata={'token_usage': ChatCompletionOutputUsage(completion_tokens=75, prompt_tokens=315, total_tokens=390), 'model': '', 'finish_reason': 'stop'}, id='run-ed5870c1-0027-476c-a74f-486323a1c17d-0')

# Text to sql

### Option : Transformers

* Load model directly

In [59]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

#https://huggingface.co/gaussalgo/T5-LM-Large-text2sql-spider


model_path = 'gaussalgo/T5-LM-Large-text2sql-spider'
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [60]:
def text_2_sql(input_text):
    """ This function takes a natural language question and a schema as input and returns the SQL query"""
    #Encode the input text
    model_inputs = tokenizer(input_text, return_tensors="pt")
    #Invoke the model
    outputs = model.generate(**model_inputs, max_length=512)
    #Decode the model outputs
    output_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return output_text

In [61]:
text_2_sql("Translate the following text to SQL: Show me the total number of rows from the inventory table")

['SELECT count(*) FROM inventory']

In [62]:
text_2_sql("Translate the following text to SQL: Show me the total number of rows from the inventory table where the product name is 'apple'")

["SELECT sum(T1.count) FROM inventory AS T1 JOIN product AS T2 ON T1.product = T2.product WHERE T2.product_name = 'apple'"]

* Use a pipeline as a high-level helper

In [63]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", model="gaussalgo/T5-LM-Large-text2sql-spider")

Device set to use cpu


In [64]:
pipe.predict("Translate the following text to SQL: Show me the total number of rows from the inventory table")

[{'generated_text': 'SELECT count(*) FROM inventory'}]

### Option 1. Pipeline (langchain_huggingface)

* Sin prompt

In [65]:

from langchain_huggingface import HuggingFacePipeline

# Example using from_model_id
hf_pipeline = HuggingFacePipeline.from_model_id(
    model_id="gaussalgo/T5-LM-Large-text2sql-spider",
    task="text2text-generation",
    pipeline_kwargs={"max_new_tokens": 500},
)

hf_pipeline.invoke("Translate the following text to SQL: Show me the total number of rows from the inventory table")



Device set to use cpu


'SELECT count(*) FROM inventory'

* Con prompt

In [66]:
from langchain_core.prompts import PromptTemplate

template = """
### System:
You are a SQL expert. Given an input question, your job is to translate the text to SQL.
### User:
{question}
### Response:
"""
prompt = PromptTemplate.from_template(template)

In [67]:
# Create a chain with a custom prompt and the HF pipeline
chain = prompt | hf_pipeline

question = "Show me the total number of rows from the inventory"

chain.invoke({"question": question})

'SELECT sum(*) FROM inventory'

### Option 2. Inference API  (langchain_huggingface)


HuggingFaceEndpoint integration of the free Serverless Endpoints API.

In [68]:
from dotenv import load_dotenv
import os

# Load the .env file
_ = load_dotenv()


* Sin prompt

In [69]:
import os
from huggingface_hub import InferenceClient

# Initialize the Hugging Face Inference Client
client = InferenceClient(
    model="gaussalgo/T5-LM-Large-text2sql-spider", 
    token=os.getenv("HUGGINGFACE_API_TOKEN")
)

# Define the text input for SQL generation
input_text = "Translate this to SQL: Show me the stock of all items."

# Call the model using `text_generation`
response = client.text_generation(input_text,)

response


'SELECT DISTINCT stock FROM inventory'

* Con prompt

In [70]:
import os
from huggingface_hub import InferenceClient

# Inicializar el cliente de Hugging Face
client = InferenceClient(
    model="gaussalgo/T5-LM-Large-text2sql-spider", 
    token=os.getenv("HUGGINGFACE_API_TOKEN")
)

# Prompt personalizado para mejorar la precisión
custom_prompt = """
You are an AI specialized in translating natural language queries into SQL queries.
The database follows standard SQL syntax.
Do not include explanations, only return the SQL query.

Translate this request into SQL:
"""

# Consulta en lenguaje natural
user_query = "Show me the stock of all items."

# Generar la entrada final para el modelo
input_text = f"{custom_prompt}\n{user_query}"

# Llamar al modelo para generar la consulta SQL
response = client.text_generation(input_text)

response


'SELECT stock FROM inventory'